In [0]:
import pandas as pd

# dbutils.fs.rm('/FileStore/tables/', recurse=True)
# dbutils.fs.mkdirs('dbfs:/FileStore/tables')
files = dbutils.fs.ls('/FileStore/tables/')
display(files)
# l = [f.name for f in files]
# print('num files: ', len(l))
# display(pd.DataFrame(l, columns=['installations']))

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas


spark = SparkSession.builder.appName("Spark DataFrames").getOrCreate()

#filtered_df = df.filter(df.installation_id is unique)
# display(df.select("installation_id").distinct())
# display(df.select("tag").distinct())

df = spark.sql("""
               select * from dev_salesforce.bronze.sf_site
            """)
display(df)

# df = spark.table('test_unify_analytics.bronze.logs_partition_1') \
#         .select('installation_id', 'tag', 'unix_timestamp', 'values') \
#         .filter((col("tag") == 242) & (col('values')[5] == 14))
# df = spark.sql("""
#                 select installation_id, tag, unix_timestamp, values
#                 from test_unify_analytics.bronze.logs_partition_1
#                 where tag = 242 AND values[5] = 14
#                 """)
# display(df)




In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.appName("Spark DataFrames").getOrCreate()

df = spark.sql("""
               select local_installation_timestamp, tag, values
               from test_unify_analytics.bronze.logs_partition_1
               where installation_id = 'a031v00000tDd2zAAC'
            """)

# df_bu_content = df.filter((df['tag'] == 242) & (df['values'][6].isNotNull())).groupBy('bin_num').count().sort('bin_num')
# df_bc_content = df.filter((df['tag'] == 362) & (df['values'][1].isNotNull())).groupBy('bin_num').count().sort('bin_num')
# df_merge = df_bu_content.join(df_bc_content, on='bin_num', how='outer').sort('bin_num')
# print('bin_update content:', df_bu_content.select(sum('count')).collect()[0][0])
# print('bin_content:', df_bc_content.select(sum('count')).collect()[0][0])
# display(df_bu_content)
# display(df_bc_content)
# display(df_merge)

# df_bu_content = df.filter((df['tag'] == 242) & (df['values'][6].isNotNull())) \
#                   .withColumn('bin_num', df['values'][0]) \
#                   .withColumn('content_code', df['values'][6]) \
#                   .orderBy(['bin_num', 'local_installation_timestamp'], ascending=True)
# df_bc_content = df.filter((df['tag'] == 362) & (df['values'][1].isNotNull())) \
#                   .withColumn('bin_num', df['values'][0]) \
#                   .withColumn('content_code', df['values'][1]) \
#                   .orderBy(['bin_num', 'local_installation_timestamp'], ascending=True)
df_cb_content = df.filter((df['tag'] == 249) & (df['values'][3].isNotNull())) \
                  .withColumn('bin_num', df['values'][1]) \
                  .withColumn('content_code', df['values'][3]) \
                  .orderBy(['bin_num', 'local_installation_timestamp'], ascending=True)

df_cb_content = df_cb_content.groupBy(['content_code']).count().sort('content_code')
# display(df_bu_content)
# display(df_cb_content)
# display(df_bc_content)
# display(df)



In [0]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np

data = tuple((row['content_code'], row['count']) for row in df_cb_content.collect())
print(len(data))
if len(data) == 0:
    dbutils.notebook.exit("no data")
    
content_code, count = zip(*data)
# print(type(data[0][0]))


kmeans = KMeans(n_clusters=2)
kmeans.fit(data)
# print(kmeans.labels_)
centers = kmeans.cluster_centers_
grouping = kmeans.labels_

fig, axs = plt.subplots(1, 2)

axs[0].scatter(content_code, count, c=kmeans.labels_, s=10)
axs[0].scatter(centers[:, 0], centers[:, 1], c='red', s=20, alpha=0.5, marker='X')
axs[0].set_yscale('log')
axs[0].set_xlabel('content code')
axs[0].set_ylabel('count')
axs[0].set_ylim(0, 10e8)
axs[0].grid(True, which='both', linestyle='--', linewidth=0.5)

axs[1].scatter(centers[:, 0], centers[:, 1], c='red', s=20, alpha=0.5, marker='X')
axs[1].set_xlabel('content code')
axs[1].set_yscale('log')
axs[1].set_ylim(0, 10e8)
axs[1].grid(True, which='both', linestyle='--', linewidth=0.5)

plt.tight_layout()
plt.show()

high_throughput = []
for i in range(0, len(content_code)):
    if grouping[i] == 1:
        high_throughput.append(content_code[i])
print('content_code:', content_code)
print('count:', count)
print('groupings:', grouping)
print('high throughput content codes:', high_throughput)

